In [ ]:
'''
kaggle - https://www.kaggle.com/code/leejin11/vit-code
'''

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [ ]:
## network class

In [ ]:
# norm -> MLP block
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim), # [b, n, dim]
            nn.Linear(dim, hidden_dim), # [b, n, hidden_dim] -> # [b, n, dim]
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim), # [b, n, hidden_dim] -> [b, n, dim]
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

# Norm -> multi head attention block
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5
        
        # norm block
        self.norm = nn.LayerNorm(dim)
        
        # softmax - dropout
        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        # query key, value 형태 벡터로 변환
        # Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V <- MULTIHEAD SELF-ATTENTION 논문 13장 
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False) # [..dim] -> [..inner_dim * 3]

        # 내부의 결과 값을 다시 dim 차원으로 변환
        self.to_out = nn.Sequential( # [..inner] -> [..dim]
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x) # [b, n, dim]

        qkv = self.to_qkv(x).chunk(3, dim = -1) # [b, n , dim] -> [b, n, inner_dim * 3]
        q, k, v = map(lambda t: rearrange(
            t, 'b n (h d) -> b h n d', h = self.heads), qkv) # [b, n, inner_dim * 3] -> [b, h, n, dim_head]
        # q, k, v = [b, h, n, dim_head]

        # attention socre 계산
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale # -> [b, h, n, dim_head] @ [b, h, n, dim_head]^T -> [b, h, n, n]

        attn = self.attend(dots) # [b, h, n, n]
        attn = self.dropout(attn) # [b, h, n, n]

        # attention 가중치를 합하기
        out = torch.matmul(attn, v) # [b, h, n, n], [b, h, n, dim_head] -> [b, h, n, dim_head]
        out = rearrange(out, 'b h n d -> b n (h d)') # [b, h, n, dim_head] -> [b, n, h * dim_head] = [b, n, inner_dim]
        return self.to_out(out) # [b, n, dim]

# transformer block
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        
        # transformer 를 여러번 시행함 
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        # Input Shape [b, n, dim]
        for attn, ff in self.layers:
            x = attn(x) + x # [b, n, dim]
            x = ff(x) + x # [b, n, dim]

        return self.norm(x) # [b, n, dim]

# image_size가 tuple(h, w) 인지 확인
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# ViT main Network
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, 
                 heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        
        # imge_size를 h, w로 분리
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        # imge_size가 patch_size보다 큰지 확인
        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        # patch의 총 개수 & 1차원에서의 dim
        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        
        # output cls 인지 mean인지 확인
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        # patch_embed에 관한 코드 ; 논문에서 Linear Projection Flattened Patches
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', 
                      p1 = patch_height, p2 = patch_width), # [b, c, h*p1, w*p2] -> [b, h*w, p1*p2*c] = [b, h*w, patch_dim] 이때 h w 는 h // patch_h
            nn.LayerNorm(patch_dim), # [b, h*w, patch_dim]
            nn.Linear(patch_dim, dim), # [b, h*w, patch_dim] -> [b, h*w, dim]
            nn.LayerNorm(dim), # [b, h*w, dim]
        )

        # [1, patch_num + 1, dim] 파라미터 생성 ; +1의 경우 cls
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        # 논문에서 transformer block
        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool

        # 빈 레이어
        self.to_latent = nn.Identity()

        # transformer 이후 mlp_head block
        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        # Input Shape [b, c, h, w]
        x = self.to_patch_embedding(img) # [b, c, h, w] -> [b, n, dim]
        b, n, _ = x.shape
        
        #  batch 크기 만큼 cls_token의 크기를 확장
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b) # [b, 1, dim]
        x = torch.cat((cls_tokens, x), dim=1) # [b, n+1, dim]
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x) # [b, n+1, dim]

        # Mean pooling을 하거나 Cls pooling을 해서 첫번 째 토큰을 가져옴
        # mean은 전체적인 정보를 ; cls는 이미지의 분류를
        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x) # [b, num_classes]
        return self.mlp_head(x) # [b, num_classes]

In [ ]:
## setting(data, prams)

In [ ]:
image_path = '/kaggle/input/imagenetmini-1000/imagenet-mini'

# vit params
vit_config = {
    "image_size": 224,
    "patch_size": 16,
    "num_classes": 100,
    "dim": 1024,
    "depth": 24,
    "heads": 16,
    "mlp_dim": 4096,
    "pool": "cls",
    "channels": 3,
    "dim_head": 64,
    "dropout": 0.1,
    "emb_dropout": 0.1
}

# load pretrain model in torchvision
from torchvision.models import vit_l_16, ViT_L_16_Weights

weights = ViT_L_16_Weights.DEFAULT
model = vit_l_16(weights=weights)